In [1]:
import json
import os
from tqdm import tqdm
from glob import glob
from streaming import MDSWriter, LocalDataset

columns = {
    'audio_filename': 'str',
    'text': 'str',
}
hashes = 'sha1', 'xxh64'

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
files = [
    '/home/husein/ssd4/pseudolabel-malaysian-youtube-whisper-large-v3/processed.jsonl',
    'force-alignment/processed.jsonl'
]

In [3]:
sebut_perkataan = glob('sebut-perkataan/*/*.wav')
sebut_perkataan = [(os.path.abspath(f), 'sebut perkataan ' + os.path.split(f)[1].replace('.wav', '')) for f in sebut_perkataan]
len(sebut_perkataan)

1463

In [4]:
import IPython.display as ipd
ipd.Audio(sebut_perkataan[0][0])

In [22]:
!rm -rf mosaic-stt

In [23]:
!wc -l {files[0]}

787242 /home/husein/ssd4/pseudolabel-malaysian-youtube-whisper-large-v3/processed.jsonl


In [24]:
!wc -l {files[1]}

881880 force-alignment/processed.jsonl


In [25]:
with MDSWriter(out='mosaic-stt', columns=columns, compression=None, hashes=hashes) as out:
    for f in files:
        folder = os.path.split(f)[0]
        with open(f) as fopen:
            for l in tqdm(fopen):
                l = json.loads(l)
                for r in l:
                    audio_filename = os.path.join(folder, r['audio_filename'])
                    audio_filename = os.path.abspath(audio_filename)
                    t = r['text'].strip()
                    before = t
                    t = ' '.join([w for w in t.split() if len(w) > 1]).strip()
                    if len(t) < 3:
                        continue
                    d = {
                        'text': t,
                        'audio_filename': audio_filename,
                    }
                    out.write(d)
        for f in sebut_perkataan:
            d = {
                'text': f[1].lower(),
                'audio_filename': f[0],
            }
            out.write(d)

787242it [03:17, 3985.13it/s]
881880it [01:17, 11396.43it/s]


In [26]:
dataset = LocalDataset(local='mosaic-stt')
len(dataset)

46226641

In [27]:
dataset[0]

{'audio_filename': '/home/husein/ssd4/pseudolabel-malaysian-youtube-whisper-large-v3/processed-audio-part3/1-179067-0.mp3',
 'text': 'muka fa'}

In [28]:
import IPython.display as ipd
ipd.Audio('/home/husein/ssd4/pseudolabel-malaysian-youtube-whisper-large-v3/processed-audio-part3/1-179067-0.mp3')